In [1]:
from mandala._next.imports import *
from mandala._next.common_imports import *

In [15]:
@op
def f(x):
    return x ** 2 + 1

@op
def g(x, y):
    return x + y

@op
def h(x):
    return x + 1


In [ ]:

storage = Storage(deps_path='__main__')

# memoizing context
with storage:
    for x in range(3):
        y = f(x)

y

In [16]:
with storage:
    for x in range(5):
        y = f(x)
        z = g(y, h(x))
    
    for y in range(15, 20):
        z = g(y, h(x))

CHANGE DETECTED in f from module __main__
Dependent components:
  Version of "f" from module "__main__" (content: 5cdf6f9924f86543b1994bd572fe7cbf, semantic: 5fcc21c8ace7aa81a5fe677db2fffc67)
===DIFF===:
 @op
 def f(x):
-    return x ** 2
+    return x ** 2 + 1
Does this change require recomputation of dependent calls? [y]es/[n]o/[a]bort 


You answered: "y"
Could not find a version for f.
Content version: 8e6645f2de90f4c8ed394367f7202896, Semantic version: 873e1f4ef59371e6e2494e204a405db0, pre_call_id: c187da4f070498870ded63c2870eb08d
Call to h with hid 44670733f0bf68e17cf52b2fb0709583 already exists.
Could not find a version for g.
Content version: f0379800b45e6f435835f724fafa5236, Semantic version: d39968dc85a0561f6dd287568aaca36a, pre_call_id: aa3569abe0ba90491c9d31bb756386be
Could not find a version for f.
Content version: 8e6645f2de90f4c8ed394367f7202896, Semantic version: 873e1f4ef59371e6e2494e204a405db0, pre_call_id: 0305217b077251114384ff98370f48a1
Call to h with hid 1c83baab01330a084f7f8c53b7f94235 already exists.
Could not find a version for g.
Content version: f0379800b45e6f435835f724fafa5236, Semantic version: d39968dc85a0561f6dd287568aaca36a, pre_call_id: e41bbe98e09ab4e974f92f17cb82bbb9
Could not find a version for f.
Content version: 8e6645f2de90f4c8ed394367f7202896, Semantic version: 873e1f4ef59371e6e2494

In [17]:
cf = storage.cf(g); cf

ComputationFrame with 3 variable(s) (33 unique refs), 1 operation(s) (15 unique calls)
Computational graph:
    output_0 = g(x=x, y=y)

In [18]:
exp_cf = cf.expand(); exp_cf

ComputationFrame with 6 variable(s) (39 unique refs), 4 operation(s) (30 unique calls)
Computational graph:
    y = h(x=x_1)
    x = f(x=x_0)
    output_0 = g(x=x, y=y)
    output_0_0 = f_0(x=x_1 | x_0)

In [19]:
exp_cf.get_df()

Extracting tuples from the computation graph:
y = h(x=x_1)
x = f(x=x_0)
output_0 = g(x=x, y=y)
output_0_0 = f_0(x=x_1 | x_0)...


,x_1,h,y,x_0,f,x,g,output_0,f_0,output_0_0
0,0,"Call(h, cid='9e1...', hid='446...')",1,0.0,"Call(f, cid='295...', hid='988...')",1,"Call(g, cid='2f6...', hid='c83...')",2.0,None,NaN
1,0,"Call(h, cid='9e1...', hid='446...')",1,0.0,"Call(f, cid='a12...', hid='fc6...')",0,"Call(g, cid='224...', hid='e52...')",1.0,None,NaN
2,3,"Call(h, cid='bdc...', hid='8fb...')",4,3.0,"Call(f, cid='e11...', hid='508...')",9,"Call(g, cid='fe9...', hid='e03...')",13.0,None,NaN
3,3,"Call(h, cid='bdc...', hid='8fb...')",4,3.0,"Call(f, cid='8fa...', hid='7b0...')",10,"Call(g, cid='ac7...', hid='7f6...')",14.0,None,NaN
4,1,"Call(h, cid='2a4...', hid='1c8...')",2,1.0,"Call(f, cid='a57...', hid='463...')",2,"Call(g, cid='2f1...', hid='526...')",4.0,None,NaN
5,1,"Call(h, cid='2a4...', hid='1c8...')",2,1.0,"Call(f, cid='6f7...', hid='100...')",1,"Call(g, cid='b6b...', hid='b6c...')",3.0,None,NaN
6,4,"Call(h, cid='b6a...', hid='079...')",5,4.0,"Call(f, cid='6c8...', hid='76a...')",16,"Call(g, cid='eed...', hid='f78...')",NaN,"Call(f, cid='64a...', hid='d50...')",17.0
7,4,"Call(h, cid='b6a...', hid='079...')",5,4.0,"Call(f, cid='6c8...', hid='76a...')",16,"Call(g, cid='eed...', hid='00f...')",NaN,"Call(f, cid='64a...', hid='d50...')",17.0
8,4,"Call(h, cid='b6a...', hid='079...')",5,NaN,None,17,"Call(g, cid='b48...', hid='a8d...')",NaN,None,NaN
9,4,"Call(h, cid='b6a...', hid='079...')",5,NaN,None,17,"Call(g, cid='b48...', hid='349...')",NaN,None,NaN


In [20]:
v = storage.get_versioner()

storage.versions(f=f)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ### Dependencies for version of function f from module __main__                                                 │
│ ### content_version_id=5cdf6f9924f86543b1994bd572fe7cbf                                                         │
│ ### semantic_version_id=5fcc21c8ace7aa81a5fe677db2fffc67                                                        │
│                                                                                                                 │
│ ################################################################################                                │
│ ### IN MODULE "__main__"                                                                                        │
│ ################################################################################                                │
│                                                                                                                 │
│ @op                                                                                                             │
│ def f(x):                                                                                                       │
│     return x ** 2                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ### Dependencies for version of function f from module __main__                                                 │
│ ### content_version_id=8e6645f2de90f4c8ed394367f7202896                                                         │
│ ### semantic_version_id=873e1f4ef59371e6e2494e204a405db0                                                        │
│                                                                                                                 │
│ ################################################################################                                │
│ ### IN MODULE "__main__"                                                                                        │
│ ################################################################################                                │
│                                                                                                                 │
│ @op                                                                                                             │
│ def f(x):                                                                                                       │
│     return x ** 2 + 1                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with storage:
    for x in range(5):
        y = f(x)
        if storage.unwrap(y) > 5:
            z = g(x, y)

In [ ]:
storage.cf(f).\
  expand().\
    eval()

In [ ]:
df.rename(columns={'output_0_0': 'output_1'})

In [ ]:
cf = storage.cf(g)
cf.expand(inplace=True)
cf.eval()

In [ ]:
cf.delete_calls()

In [ ]:
@op
def avg_items(xs: MList[int]) -> float:
    return sum(xs) / len(xs)

@op
def get_xs(n: int) -> MList[int]:
    return list(range(n))

storage = Storage()

with storage:
    xs = get_xs(10)
    for i in range(2, 10, 2):
        avg = avg_items(xs[:i])

In [ ]:
cf = storage.cf(avg_items).expand()
cf.eval()

In [ ]:
storage = Storage()

@op
def inc(eggs):
    print(f'Executing inc with eggs={eggs}')
    return eggs + 1

with storage:
    spam = inc(eggs=42)

@op
def make_breakfast(spam, eggs):
    print(f'Executing make_breakfast with spam={spam} and eggs={eggs}')
    return spam + eggs

with storage:
    eggs = 42
    spam = inc(eggs=eggs)
    more_spam = inc(eggs=spam)
    breakfast = make_breakfast(spam=spam, eggs=eggs)
    breakfast_2 = make_breakfast(spam=more_spam, eggs=eggs)


cf = storage.cf(make_breakfast)
cf

cf = cf.expand()
cf.eval()

In [ ]:
storage = Storage()

### a few simple ops covering all cases we care about
@op
def f(x: int) -> int:
    return x + 1

@op
def g(x: int, y: int) -> Tuple[int, int]: # multiple inputs and outputs
    return x + 2, y ** 2

@op 
def inc_list(things: MList[int]) -> MList[int]: # lists as inputs and outputs
    return [x + 1 for x in things]

@op
def inc_dict(things: MDict[str, int]) -> MDict[str, int]: # dicts as inputs and outputs
    return {k: v + 1 for k, v in things.items()}

@op
def fancy_signature(x: int, *args, y: bool = False, **kwargs) -> Tuple[int, List[int], MList[int]]: # fancy signatures
    return x, [int(a) for a in args], [int(v) for v in kwargs.values()]

In [ ]:
### simple workflow
with storage:
    x = f(1)
    y, t = g(x, 2)
    z = inc_list([1, 2])
    z = inc_list([x, y])
    z = inc_list([1, x])
    w = inc_dict({'x': x, 'y': y})
    a = fancy_signature(1, 2, x, y=True, z=4, w=5)